#### Game Setup

- Player 1 is 'X', indexed by 0. It is also status of board when Player 1 Wins.
- Player 2 is 'O', indexed by 1. It is also status of board when Player 2 Wins. 
- An unfilled spot is ".", indexed by 2. And 2 is also the status when game is incomplete.
- 3 indicates the game is a tie. 

In [7]:
import numpy as np
import pandas as pd
import random
from numba import jit
import warnings
warnings.filterwarnings('ignore')

def print_board(board, depth = 0):
    mapping = {0:'x',1:'o',2:'.'}
    x = list(map(lambda x: mapping[x] if x in mapping else x, board))
    print('\n')
    print('\t'*depth,x[0],'|',x[1],'|',x[2])
    print('\t'*depth,'---------')
    print('\t'*depth,x[3],'|',x[4],'|',x[5])
    print('\t'*depth,'---------')
    print('\t'*depth,x[6],'|',x[7],'|',x[8])
    
@jit
def permissible_actions(x):
    return np.where(x == 2)[0]    

@jit
def random_action(board):
    return np.random.choice(permissible_actions(board))

@jit
def check_status(board):
    # Define all possible winning combinations
    winning_combinations = [
        [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Rows
        [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Columns
        [0, 4, 8], [2, 4, 6]             # Diagonals
    ]
    
    # Check each winning combination for a win
    for combo in winning_combinations:
        if board[combo[0]] == board[combo[1]] == board[combo[2]] and board[combo[0]] != 2:
            return board[combo[0]]
    
    # Check for a draw
    if 2 not in board:
        return 3
    
    return 2

    
@jit
def visualize_game(list_x):
    for x in list_x:
        print_board(x[-1])

def update_board(board,action_idx):
    player_idx, opponent_idx = who_to_move(board)
    board_new = board.copy()
    if player_idx == 0:
        board_new[action_idx] = 0
    else:
        board_new[action_idx] = 1
    return board_new


@jit
def who_to_move(board):
    if np.sum(np.where(board==0,1,0))>np.sum(np.where(board==1,1,0)):
        return 1,0
    else:
        return 0,1

@jit
def play_random_game(verbose = 0):
    board = np.array([2,2,2,2,2,2,2,2,2])
    game_history = []
    turn_count = 0
    status = 2
    while True:
        player_idx, opponent_idx = who_to_move(board)
        status = check_status(board)
        if status != 2:
            status_final = status
            break
        turn_count += 1
        action_idx = random_action(board)
        board_new = update_board(board,action_idx)
        game_history.append([board, status, turn_count, player_idx, opponent_idx, action_idx, board_new])
        board = board_new.copy()
    if verbose == 1:
        visualize_game(game_history)
    return game_history, status

@jit
def generate_random_board():
    game_history, status = play_random_game()
    boards = [events[-1] for events in game_history]
    return random.choice(boards)

@jit
def simulate_games(N):
    data = [] # board, action, win/loss
    for i in range(N):
        game_history, status_final = play_random_game()
        for event in game_history:
            board, status, turn_count, player_idx, opponent_idx, action_idx, board_new = event
            if status_final == 3:
                score = 0
            elif status_final == player_idx:
                score = +1
            else:
                score = -1
            data.append([i, ''.join(map(str, board)), turn_count, player_idx, opponent_idx, action_idx, score, status_final])
    return data

#### Random X vs Random O

In [3]:
import pandas as pd
data = simulate_games(10000)
df = pd.DataFrame(data, columns = ['Game','Board', 'Turn','Player','Opponent','Action', 'Score', 'Result'])
df.head()

,Game,Board,Turn,Player,Opponent,Action,Score,Result
0,0,222222222,1,0,1,5,-1,1
1,0,222220222,2,1,0,4,1,1
2,0,222210222,3,0,1,3,-1,1
3,0,222010222,4,1,0,8,1,1
4,0,222010221,5,0,1,1,-1,1


In [4]:
df.Result.value_counts()/df.shape[0]

0    0.562177
1    0.287064
3    0.150760
Name: Result, dtype: float64

#### Minmax Algorithm

- generate_childs: find all children boards
- maximize: find the best value of an action given childs
- minimize: find the lowest value of an action given childs
- value_terminal: returns the value of a terminal node / board

In [8]:
def generate_child(board, visualize=False):
    possible_actions = permissible_actions(board)
    childs = []
    for action_idx in possible_actions:
        childs.append(update_board(board, action_idx))
        if visualize==True:
            print_board(update_board(board, action_idx))
    return childs

def maximize(board,move_idx=False):
    player_idx, opponent_idx = who_to_move(board)
    status = check_status(board)
    if status == player_idx:
        return 1 # maximizer won, minimizer lost
    elif status == opponent_idx:
        return -1 # minimizer won, maximizer lost
    elif status == 3:
        return 0
    else:
        childs = generate_child(board)
        values = [-maximize(child, move_idx=False) for child in childs]
        max_index = np.argmax(values)
        max_value = values[max_index]
        if move_idx:
            return max_value, permissible_actions(board)[max_index]
        else:
            return max_value
    
def minimax_action(board):
    value, idx = maximize(board, True)
    return idx

@jit
def play_minimax_game(verbose = 0):
    board = np.array([2,2,2,2,2,2,2,2,2])
    game_history = []
    turn_count = 0
    status = 2
    while True:
        player_idx, opponent_idx = who_to_move(board)
        status = check_status(board)
        if status != 2:
            status_final = status
            break
        turn_count += 1
        action_idx = minimax_action(board)
        board_new = update_board(board,action_idx)
        game_history.append([board, status, turn_count, player_idx, opponent_idx, action_idx, board_new])
        board = board_new.copy()
    if verbose == 1:
        visualize_game(game_history)
    return game_history, status

In [9]:
play_minimax_game(1)



 x | . | .
 ---------
 . | . | .
 ---------
 . | . | .


 x | . | .
 ---------
 . | o | .
 ---------
 . | . | .


 x | x | .
 ---------
 . | o | .
 ---------
 . | . | .


 x | x | o
 ---------
 . | o | .
 ---------
 . | . | .


 x | x | o
 ---------
 . | o | .
 ---------
 x | . | .


 x | x | o
 ---------
 o | o | .
 ---------
 x | . | .


 x | x | o
 ---------
 o | o | x
 ---------
 x | . | .


 x | x | o
 ---------
 o | o | x
 ---------
 x | o | .


 x | x | o
 ---------
 o | o | x
 ---------
 x | o | x


([[array([2, 2, 2, 2, 2, 2, 2, 2, 2]),
   2,
   1,
   0,
   1,
   0,
   array([0, 2, 2, 2, 2, 2, 2, 2, 2])],
  [array([0, 2, 2, 2, 2, 2, 2, 2, 2]),
   2,
   2,
   1,
   0,
   4,
   array([0, 2, 2, 2, 1, 2, 2, 2, 2])],
  [array([0, 2, 2, 2, 1, 2, 2, 2, 2]),
   2,
   3,
   0,
   1,
   1,
   array([0, 0, 2, 2, 1, 2, 2, 2, 2])],
  [array([0, 0, 2, 2, 1, 2, 2, 2, 2]),
   2,
   4,
   1,
   0,
   2,
   array([0, 0, 1, 2, 1, 2, 2, 2, 2])],
  [array([0, 0, 1, 2, 1, 2, 2, 2, 2]),
   2,
   5,
   0,
   1,
   6,
   array([0, 0, 1, 2, 1, 2, 0, 2, 2])],
  [array([0, 0, 1, 2, 1, 2, 0, 2, 2]),
   2,
   6,
   1,
   0,
   3,
   array([0, 0, 1, 1, 1, 2, 0, 2, 2])],
  [array([0, 0, 1, 1, 1, 2, 0, 2, 2]),
   2,
   7,
   0,
   1,
   5,
   array([0, 0, 1, 1, 1, 0, 0, 2, 2])],
  [array([0, 0, 1, 1, 1, 0, 0, 2, 2]),
   2,
   8,
   1,
   0,
   7,
   array([0, 0, 1, 1, 1, 0, 0, 1, 2])],
  [array([0, 0, 1, 1, 1, 0, 0, 1, 2]),
   2,
   9,
   0,
   1,
   8,
   array([0, 0, 1, 1, 1, 0, 0, 1, 0])]],
 3)

In [14]:
@jit
def simulate_minimax_games(N):
    data = [] # board, action, win/loss
    for i in range(N):
        game_history, status_final = play_minimax_game()
        for event in game_history:
            board, status, turn_count, player_idx, opponent_idx, action_idx, board_new = event
            if status_final == 3:
                score = 0
            elif status_final == player_idx:
                score = +1
            else:
                score = -1
            data.append([i, ''.join(map(str, board)), turn_count, player_idx, opponent_idx, action_idx, score, status_final])
    return data

In [237]:
data = simulate_minimax_games(10)

#### Alpha Beta Pruning

- depth: is how deep the algorithm will perform the search. In tic tac toe this is usually till the game ends. 
- alpha: Alpha is the best value currently known to the maximizing player (AI) on the path from the root to the current node. It represents the minimum score that the maximizing player is assured of if they follow the path through this node. The initial value of alpha is negative infinity (indicating the worst possible score for the maximizing player). It increases during the search. It does not make sense to 
- beta: Beta is the best value currently known to the minimizing player (opponent) on the path from the root to the current node. It represents the maximum score that the minimizing player is assured of if they follow the path through this node. The initial value of beta is positive infinity (indicating the best possible score for the minimizing player). It decreases during the search. 
- player_idx: index of player making the move i.e. the maximizing player.
- opponent_idx: index of player making the move afterwards i.e. the minimizing player. 
- Pruning: if beta of any node falls below alpha of the node, it means that 

In [10]:
def minimax_alpha_beta(board, depth=0, maximizer = True, alpha = np.inf, beta = -np.inf, move_idx = False, pruning = True):
    player_idx, opponent_idx = who_to_move(board)
    status = check_status(board)
    #print_board(board, depth)
    
    if maximizer:
        if status == player_idx:
            evaluation = 1
        elif status == opponent_idx:
            evaluation = -1
        elif status == 3:
            evaluation = 0
        else: 
            evaluation = -np.inf
            childs = generate_child(board)
            for idx, child in enumerate(childs):
                value = minimax_alpha_beta(child,depth=depth+1,maximizer = False, alpha = alpha, beta = beta, move_idx=False)
                if value>=evaluation:
                    evaluation = value
                    alpha = value
                    evaluation_idx = idx
                if (beta <= alpha) and (pruning == True):
                    #print('Pruned!')
                    break
        #print('------->'*depth,'Maximizer Eval:',evaluation)
        if move_idx == True:
            return evaluation_idx
        return evaluation
        
    else:
        if status == player_idx:
            evaluation = -1
        elif status == opponent_idx:
            evaluation = +1
        elif status == 3:
            evaluation = 0
        else:
            evaluation = np.inf
            childs = generate_child(board)
            for idx, child in enumerate(childs):
                value = minimax_alpha_beta(child, depth=depth+1,maximizer = True, alpha = alpha, beta = beta, move_idx=False)
                if value<=evaluation:
                    evaluation = value
                    beta = value
                    evaluation_idx = idx
                if (beta <= alpha) and (pruning == True):
                    # print('Pruned!')
                    break
        #print('------->'*depth,'Minimizer Eval:',evaluation)
        if move_idx == True:
            return evaluation_idx
        return evaluation

In [25]:
test = generate_random_board()
#test = np.array([2,2,2,2,2,2,2,2,2])
print_board(test,1)



	 . | . | .
	 ---------
	 . | o | .
	 ---------
	 o | x | x


In [28]:
minimax_alpha_beta(test)

1

In [134]:
%timeit minimax_alpha_beta(test, pruning=True)

72.3 µs ± 5.78 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [135]:
%timeit minimax_alpha_beta(test, pruning=False)

660 µs ± 22.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
